<a href="https://colab.research.google.com/github/RoshiEttehadi/AI-CNN-SKINCANCER/blob/main/Model_Presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def create_and_load_model(weights_path):
    """
    Recreate the model architecture and load weights
    """
    # Create base model
    base_model = VGG16(
        include_top=False,
        input_shape=(224, 224, 3),
        weights='imagenet'
    )
    base_model.trainable = False

    # Recreate the model architecture
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)

    # Load weights if they exist
    if os.path.exists(weights_path):
        try:
            model.load_weights(weights_path)
            print("Model weights loaded successfully!")
        except Exception as e:
            print(f"Error loading weights: {str(e)}")
    else:
        raise FileNotFoundError(f"Weights file not found at {weights_path}")

    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

def predict_single_image(model, image_path):
    """
    Make prediction for a single image
    """
    try:
        # Load and preprocess the image
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img_array, verbose=0)
        probability = prediction[0][0]
        classification = 'Malignant' if probability > 0.5 else 'Benign'

        return classification, probability
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None, None

def test_model_on_folder(model, test_folder):
    """
    Test the model on all images in a folder
    """
    if not os.path.exists(test_folder):
        raise FileNotFoundError(f"Test folder not found at {test_folder}")

    results = []
    for filename in os.listdir(test_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(test_folder, filename)
            classification, probability = predict_single_image(model, image_path)

            if classification and probability is not None:
                results.append({
                    'image': filename,
                    'classification': classification,
                    'probability': float(probability)
                })

    return results

def main():
    # Define paths
    DRIVE_BASE = '/content/gdrive/MyDrive'
    MODEL_PATH = os.path.join(DRIVE_BASE, 'models')
    WEIGHTS_PATH = os.path.join(MODEL_PATH, 'model_weights.weights.h5')
    TEST_PATH = os.path.join(DRIVE_BASE, 'datasets', 'Test')

    try:
        # Create model and load weights
        print("Loading model...")
        model = create_and_load_model(WEIGHTS_PATH)

        # Test on folder if it exists
        if os.path.exists(TEST_PATH):
            print("\nTesting model on folder...")
            results = test_model_on_folder(model, TEST_PATH)

            # Print results
            print("\nResults:")
            for result in results:
                print(f"Image: {result['image']}")
                print(f"Classification: {result['classification']}")
                print(f"Probability: {result['probability']:.2%}")
                print("-" * 50)
        else:
            print(f"Test folder not found at {TEST_PATH}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Loading model...
Model weights loaded successfully!

Testing model on folder...

Results:
Image: ISIC_4576846.jpg
Classification: Malignant
Probability: 81.70%
--------------------------------------------------
Image: ISIC_7545363.jpg
Classification: Malignant
Probability: 70.00%
--------------------------------------------------
Image: ISIC_7547657.jpg
Classification: Benign
Probability: 28.39%
--------------------------------------------------
Image: ISIC_4575019.jpg
Classification: Benign
Probability: 42.05%
--------------------------------------------------
Image: ISIC_2412113.jpg
Classification: Malignant
Probability: 72.59%
--------------------------------------------------
Image: ISIC_2427899.jpg
Classification: Malignant
Probability: 90.43%
--------------------------------------------------
Image: ISIC_0046773.jpg
Classification: Malignant
Probability: 90.38%
--------------------------------------------------
Image: ISIC_0046741.jpg
Classification: Malignant
Probability: 71.60%